# Face Masks Filter

In [ ]:
import cv2
import numpy as np

def add_filter(filter_mask, title, height_stretch_factor, height_translation_factor):
    face_cascade = cv2.CascadeClassifier('../Data/Haarcascades/haarcascade_frontalface_default.xml')
    h_mask, w_mask, _ = filter_mask.shape
    cap = cv2.VideoCapture(0)
    scaling_factor = 0.5
    while True:
        ret, frame = cap.read()
        frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in face_rects:
            if h > 0 and w > 0:
                h, w = int(height_stretch_factor * h), int(1.0*w)
                y -= int(height_translation_factor * h)
                frame_roi = frame[y:y+h, x:x+w]
                face_mask_small = cv2.resize(filter_mask, (w, h), interpolation=cv2.INTER_AREA)
                gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_BINARY_INV)
                masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                mask_inv = cv2.bitwise_not(mask)
                masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                cv2.imshow('Gray', gray_mask)
                cv2.imshow('Thresh', mask)
                cv2.imshow('Thresh_INV', mask_inv)
                cv2.imshow('Masked Face', masked_face)
                cv2.imshow('Masked Frame', masked_frame)
                frame[y:y+h, x:x+w] = cv2.add(masked_face, masked_frame)
        cv2.imshow(title, frame)
        c = cv2.waitKey(1)
        if c == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
    

filters = {'Bane'    : ('bane_mask', 'Bane', 1.2, 0.02),
           'Vendetta': ('vendetta_mask', 'Vendetta', 1.3, 0.05),
           'Skull'    : ('skull_mask', 'Skull', 1.3, 0.1)}

mask_path, title, h_s_f, h_t_f = filters['Vendetta']
add_filter(cv2.imread('../Data/Images/filter_sprites/' + mask_path +'.png'), title, h_s_f, h_t_f)

# Resources

for more stablity use Dlib instead of OpenCV
https://github.com/coffe2code/Face-Filter